_Notebook assembled by Prof. Paul Waddell, Fall 2020_

## Interactive Mapping With Deck.gl from Uber

Installation steps:
in the terminal window (command shell), before running jupyter notebook, do the following

pip install pydeck

jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck

jupyter nbextension enable --sys-prefix --py pydeck

You will also need to create an environment variable on your computer to store your Mapbox API key.  The variable needs to be named exacty as 'MAPBOX_API_KEY' and should be set to your own API key.  The code below will probably run for the deck component but won't load any Mapbox background maps if you don't do this.

Because each operating system and version handle this differently, it is left to your own efforts to find out how to store this environment variable.  But here are a couple of links to look at for helpful hints on certain OS versions:

For Mac Catalina users: https://medium.com/@grantbrits/editing-path-variable-in-catalina-mac-osx-5532ba447af9
(the discussion is about PATH variables, but our MAPBOX_API_KEY variable works the same way)

There might be a better/more current link for windows but this one might help, and it covers Linux and older OSX versions:
https://www.twilio.com/blog/2017/01/how-to-set-environment-variables.html

In [ ]:
import pydeck as pdk

UK_ACCIDENTS_DATA = 'https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/3d-heatmap/heatmap-data.csv'

layer = pdk.Layer(
    'HexagonLayer',  # `type` positional argument is here
    UK_ACCIDENTS_DATA,
    get_position=['lng', 'lat'],
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,
    coverage=1)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-1.415,
    latitude=52.2323,
    zoom=6,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36)

# Combined all of it and render a viewport
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html('hexagon-example.html')

Try changing type above to ScatterplotLayer and add some ScatterplotLayer attributes, like get_fill_color and get_radius:

In [ ]:
import pydeck as pdk

UK_ACCIDENTS_DATA = 'https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/3d-heatmap/heatmap-data.csv'

layer = pdk.Layer(
    'ScatterplotLayer',     # Change the `type` positional argument here
    UK_ACCIDENTS_DATA,
    get_position=['lng', 'lat'],
    auto_highlight=True,
    get_radius=1000,          # Radius is given in meters
    get_fill_color=[180, 0, 200, 140],  # Set an RGBA value for fill
    pickable=True)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-1.415,
    latitude=52.2323,
    zoom=6,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36)

# Combined all of it and render a viewport
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html('hexagon-example.html')



In [ ]:
"""
GeoJsonLayer
===========

Property values in Vancouver, Canada, adapted from the deck.gl example pages. Input data is in a GeoJSON format.
"""

import pydeck

DATA_URL = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/geojson/vancouver-blocks.json"
LAND_COVER = [[[-123.0, 49.196], [-123.0, 49.324], [-123.306, 49.324], [-123.306, 49.196]]]

INITIAL_VIEW_STATE = pydeck.ViewState(latitude=49.254, longitude=-123.13, zoom=11, max_zoom=16, pitch=45, bearing=0)

polygon = pydeck.Layer(
    "PolygonLayer",
    LAND_COVER,
    stroked=False,
    # processes the data as a flat longitude-latitude pair
    get_polygon="-",
    get_fill_color=[0, 0, 0, 20],
)

geojson = pydeck.Layer(
    "GeoJsonLayer",
    DATA_URL,
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation="properties.valuePerSqm / 20",
    get_fill_color="[255, 255, properties.growth * 255]",
    get_line_color=[255, 255, 255],
)

r = pydeck.Deck(layers=[polygon, geojson], initial_view_state=INITIAL_VIEW_STATE)

r.to_html("geojson_layer.html")

In [ ]:
"""
ArcLayer
========

Map of commutes to work within a segment of downtown San Francisco using a deck.gl ArcLayer.

Green indicates a start point, and red indicates the destination.

The data is collected by the US Census Bureau and viewable in the 2017 LODES data set: https://lehd.ces.census.gov/data/
"""

import pydeck as pdk
import pandas as pd

DATA_URL = "https://raw.githubusercontent.com/ajduberstein/sf_public_data/master/bay_area_commute_routes.csv"
# A bounding box for downtown San Francisco, to help filter this commuter data
DOWNTOWN_BOUNDING_BOX = [
    -122.43135291617365,
    37.766492914983864,
    -122.38706428091974,
    37.80583561830737,
]


def in_bounding_box(point):
    """Determine whether a point is in our downtown bounding box"""
    lng, lat = point
    in_lng_bounds = DOWNTOWN_BOUNDING_BOX[0] <= lng <= DOWNTOWN_BOUNDING_BOX[2]
    in_lat_bounds = DOWNTOWN_BOUNDING_BOX[1] <= lat <= DOWNTOWN_BOUNDING_BOX[3]
    return in_lng_bounds and in_lat_bounds


df = pd.read_csv(DATA_URL)
# Filter to bounding box
df = df[df[["lng_w", "lat_w"]].apply(lambda row: in_bounding_box(row), axis=1)]

GREEN_RGB = [0, 255, 0, 40]
RED_RGB = [240, 100, 0, 40]

# Specify a deck.gl ArcLayer
arc_layer = pdk.Layer(
    "ArcLayer",
    data=df,
    get_width="S000 * 2",
    get_source_position=["lng_h", "lat_h"],
    get_target_position=["lng_w", "lat_w"],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=GREEN_RGB,
    pickable=True,
    auto_highlight=True,
)

view_state = pdk.ViewState(
    latitude=37.7576171,
    longitude=-122.5776844,
    bearing=45,
    pitch=50,
    zoom=8,
)


TOOLTIP_TEXT = {"html": "{S000} jobs <br /> Home of commuter in red; work location in green"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT)
r.to_html("arc_layer.html")

In [ ]:
"""
GreatCircleLayer
================

Plot of direct flights to and from San Francisco International Airport.

Origin is in green; destinations are in blue.

Adapted from the deck.gl documentation.
"""

import pydeck as pdk
import pandas as pd

GREAT_CIRCLE_LAYER_DATA = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/flights.json"  # noqa

df = pd.read_json(GREAT_CIRCLE_LAYER_DATA)

# Use pandas to prepare data for tooltip
df["from_name"] = df["from"].apply(lambda f: f["name"])
df["to_name"] = df["to"].apply(lambda t: t["name"])

# Define a layer to display on a map
layer = pdk.Layer(
    "GreatCircleLayer",
    df,
    pickable=True,
    get_stroke_width=12,
    get_source_position="from.coordinates",
    get_target_position="to.coordinates",
    get_source_color=[64, 255, 0],
    get_target_color=[0, 128, 200],
    auto_highlight=True,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=50, longitude=-40, zoom=1, bearing=0, pitch=0)

# Render
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "{from_name} to {to_name}"},
)
r.picking_radius = 10

r.to_html("great_circle_layer.html")

In [ ]:
"""
H3ClusterLayer
==============

Data grouped by H3 geohash, as an example of one of the geohash schemes supported by pydeck.

This layer joins contiguous regions into the same color. Data format is as follows:

    [
      {
        "mean": 73.333,
        "count": 440,
        "hexIds": [
          "88283082b9fffff",
          "88283082b1fffff",
          "88283082b5fffff",
          "88283082b7fffff",
          "88283082bbfffff",
          "882830876dfffff"
        ]
      },
      {
        "mean": 384.444,
        "count": 3460,
        "hexIds": [
          "8828308281fffff",
          "8828308287fffff",
          "88283082abfffff",
          "88283082a3fffff",
          "8828308289fffff",
          "88283082c7fffff",
          "88283082c3fffff",
          "88283082c1fffff",
          "88283082d5fffff"
        ]
      },
      ...

If you'd simply like to plot a value at an H3 hex ID, see the H3HexagonLayer.

This example is adapted from the deck.gl documentation.
"""

import pydeck as pdk
import pandas as pd

H3_CLUSTER_LAYER_DATA = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/sf.h3clusters.json"  # noqa

df = pd.read_json(H3_CLUSTER_LAYER_DATA)

# Define a layer to display on a map
layer = pdk.Layer(
    "H3ClusterLayer",
    df,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=False,
    get_hexagons="hexIds",
    get_fill_color="[255, (1 - mean / 500) * 255, 0]",
    get_line_color=[255, 255, 255],
    line_width_min_pixels=2,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=11, bearing=0, pitch=30)


# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "Density: {mean}"})
r.to_html("h3_cluster_layer.html")

In [ ]:
"""
HeatmapLayer
===========

Location of livestock raised in New Mexico in the United States in 2006,
via the United Nations and FAOSTAT, with the source data viewable here: http://www.fao.org/faostat/en/

Locations for poultry are viewable in blue and cattle are in orange.

Overlaid with the satellite imagery from Mapbox to highlight the how terrain affects agriculture.
"""

import pandas as pd
import pydeck as pdk

CATTLE_DATA = "https://raw.githubusercontent.com/ajduberstein/geo_datasets/master/nm_cattle.csv"
POULTRY_DATA = "https://raw.githubusercontent.com/ajduberstein/geo_datasets/master/nm_chickens.csv"


HEADER = ["lng", "lat", "weight"]
cattle_df = pd.read_csv(CATTLE_DATA, header=None).sample(frac=0.5)
poultry_df = pd.read_csv(POULTRY_DATA, header=None).sample(frac=0.5)

cattle_df.columns = HEADER
poultry_df.columns = HEADER

COLOR_BREWER_BLUE_SCALE = [
    [240, 249, 232],
    [204, 235, 197],
    [168, 221, 181],
    [123, 204, 196],
    [67, 162, 202],
    [8, 104, 172],
]


view = pdk.data_utils.compute_view(cattle_df[["lng", "lat"]])
view.zoom = 6

cattle = pdk.Layer(
    "HeatmapLayer",
    data=cattle_df,
    opacity=0.9,
    get_position=["lng", "lat"],
    aggregation='"MEAN"',
    color_range=COLOR_BREWER_BLUE_SCALE,
    threshold=1,
    get_weight="weight",
    pickable=True,
)

poultry = pdk.Layer(
    "HeatmapLayer",
    data=poultry_df,
    opacity=0.9,
    get_position=["lng", "lat"],
    threshold=0.75,
    aggregation='"MEAN"',
    get_weight="weight",
    pickable=True,
)


r = pdk.Deck(
    layers=[cattle, poultry],
    initial_view_state=view,
    tooltip={"text": "Concentration of cattle in blue, concentration of poultry in orange"},
)

r.to_html("heatmap_layer.html")

In [ ]:
"""
IconLayer
=========

Location of biergartens in Germany listed on OpenStreetMap as of early 2020.
"""

import pydeck as pdk
import pandas as pd


# Data from OpenStreetMap, accessed via osmpy
DATA_URL = "https://raw.githubusercontent.com/ajduberstein/geo_datasets/master/biergartens.json"
ICON_URL = "https://upload.wikimedia.org/wikipedia/commons/c/c4/Projet_bi%C3%A8re_logo_v2.png"

icon_data = {
    # Icon from Wikimedia, used the Creative Commons Attribution-Share Alike 3.0
    # Unported, 2.5 Generic, 2.0 Generic and 1.0 Generic licenses
    "url": ICON_URL,
    "width": 242,
    "height": 242,
    "anchorY": 242,
}

data = pd.read_json(DATA_URL)
data["icon_data"] = [icon_data] * len(data)
# for i, row in data.iterrows():
#     data.loc[i, "icon_data"] = icon_data

view_state = pdk.data_utils.compute_view(data[["lon", "lat"]], 0.1)

icon_layer = pdk.Layer(
    type="IconLayer",
    data=data,
    get_icon="icon_data",
    get_size=4,
    size_scale=15,
    get_position=["lon", "lat"],
    pickable=True,
)

r = pdk.Deck(layers=[icon_layer], initial_view_state=view_state, tooltip={"text": "{tags}"})
r.to_html("icon_layer.html")

In [ ]:
"""
ScatterplotLayer
================

Plot of the number of exits for various subway stops within San Francisco, California.

Adapted from the deck.gl documentation.
"""

import pydeck as pdk
import pandas as pd
import math

SCATTERPLOT_LAYER_DATA = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/bart-stations.json"
df = pd.read_json(SCATTERPLOT_LAYER_DATA)

# Use pandas to calculate additional data
df["exits_radius"] = df["exits"].apply(lambda exits_count: math.sqrt(exits_count))

# Define a layer to display on a map
layer = pdk.Layer(
    "ScatterplotLayer",
    df,
    pickable=True,
    opacity=0.8,
    stroked=True,
    filled=True,
    radius_scale=6,
    radius_min_pixels=1,
    radius_max_pixels=100,
    line_width_min_pixels=1,
    get_position="coordinates",
    get_radius="exits_radius",
    get_fill_color=[255, 140, 0],
    get_line_color=[0, 0, 0],
)

# Set the viewport location
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=10, bearing=0, pitch=0)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "{name}\n{address}"})
r.to_html("scatterplot_layer.html")

In [ ]:
"""
ScreenGridLayer
===============

Locations of bike parking within San Francisco. Aggregation level varies depending on the viewport of the user.

Adapted from the deck.gl documentation.
"""

import pydeck as pdk
import pandas as pd

SCREEN_GRID_LAYER_DATA = (
    "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/sf-bike-parking.json"  # noqa
)
df = pd.read_json(SCREEN_GRID_LAYER_DATA)

# Define a layer to display on a map
layer = pdk.Layer(
    "ScreenGridLayer",
    df,
    pickable=False,
    opacity=0.8,
    cell_size_pixels=50,
    color_range=[
        [0, 25, 0, 25],
        [0, 85, 0, 85],
        [0, 127, 0, 127],
        [0, 170, 0, 170],
        [0, 190, 0, 190],
        [0, 255, 0, 255],
    ],
    get_position="COORDINATES",
    get_weight="SPACES",
)

# Set the viewport location
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=11, bearing=0, pitch=0)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html("screengrid_layer.html")

In [ ]:
"""
TerrainLayer
===========

Extruded terrain using AWS Open Data Terrain Tiles and Mapbox Satellite imagery
"""

import pydeck as pdk
import os
import json

# Import Mapbox API Key from environment
with open("data/mapbox_api_key.json") as f:
    MAPBOX_API_KEY = json.load(f)
# MAPBOX_API_KEY = os.environ["MAPBOX_API_KEY"]

# AWS Open Data Terrain Tiles
TERRAIN_IMAGE = '"https://s3.amazonaws.com/elevation-tiles-prod/terrarium/{z}/{x}/{y}.png"'

# Define how to parse elevation tiles
ELEVATION_DECODER = {"rScaler": 256, "gScaler": 1, "bScaler": 1 / 256, "offset": -32768}

SURFACE_IMAGE = '"https://api.mapbox.com/v4/mapbox.satellite/{{z}}/{{x}}/{{y}}@2x.png?access_token={}"'.format(
    MAPBOX_API_KEY
)

terrain_layer = pdk.Layer(
    "TerrainLayer", data=None, elevation_decoder=ELEVATION_DECODER, texture=SURFACE_IMAGE, elevation_data=TERRAIN_IMAGE
)

view_state = pdk.ViewState(latitude=46.24, longitude=-122.18, zoom=11.5, bearing=140, pitch=60)

r = pdk.Deck(terrain_layer, initial_view_state=view_state)

r.to_html("terrain_layer.html")

In [ ]:
"""
TripsLayer
==========

Plot of a single vehicle trip within San Francisco, fading in from the origin.

Adapted from a deck.gl documentation example.
"""

import pydeck as pdk
import pandas as pd

TRIPS_LAYER_DATA = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/sf.trips.json"  # noqa

df = pd.read_json(TRIPS_LAYER_DATA)

df["coordinates"] = df["waypoints"].apply(lambda f: [item["coordinates"] for item in f])
df["timestamps"] = df["waypoints"].apply(lambda f: [item["timestamp"] - 1554772579000 for item in f])

df.drop(["waypoints"], axis=1, inplace=True)

layer = pdk.Layer(
    "TripsLayer",
    df,
    get_path="coordinates",
    get_timestamps="timestamps",
    get_color=[253, 128, 93],
    opacity=0.8,
    width_min_pixels=5,
    rounded=True,
    trail_length=600,
    current_time=500,
)

view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=11, bearing=0, pitch=45)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html("trips_layer.html", notebook_display=True)